In [3]:
import torch
import pickle

In [8]:
from pathlib import Path
root_path = Path("/root/workspace/")
with open(root_path / "training_triples.pkl", "rb") as f:
    training_triples = pickle.load(f)
with open(root_path/"testing_triples.pkl", "rb") as f:
    testing_triples = pickle.load(f)
with open(root_path / "validation_triples.pkl", "rb") as f:
    validation_triples_t = pickle.load(f)

# Lessons
1. Convert to embeddings late in the data loader 
2. Use packed padding if your have sequence lengths that can be sorted.

In [18]:
training_triples[0]

(19699,
 array([-0.45027733, -1.1089361 , -0.7485446 ,  0.29338717, -1.1459185 ,
        -1.5272202 , -0.74012953, -0.30964288,  0.21130872,  0.35100073,
         1.0060655 ,  0.20887893, -0.24708146, -0.731051  , -1.4602807 ,
         0.34196234,  0.1258754 , -0.38644987,  0.4168012 , -0.5980486 ,
         0.4608552 , -0.68917155,  0.09706956, -0.32858768, -2.4480584 ,
        -0.6501933 ,  0.6384356 , -2.113722  ,  0.46189746,  1.6031395 ,
        -0.7843428 , -0.94217765,  0.07016011,  0.06641492, -2.471242  ,
         1.3342896 ,  0.41738454, -1.3223062 , -3.1056533 , -0.44302684,
         1.8227277 , -0.5422932 ,  1.686431  ,  0.74762636, -0.5201043 ,
         0.24003752, -2.5744867 , -2.4474618 ,  0.31771022, -1.8747869 ,
        -0.329535  , -1.818454  ,  0.7101862 , -1.9354166 ,  0.5667974 ,
         0.419285  , -1.0025371 , -1.5425253 ,  2.3222175 ,  1.239261  ,
         0.09211183, -0.49247086, -1.3927358 , -0.1261586 ,  1.4260572 ,
        -1.3440863 ,  0.27712378, -0.728670

In [10]:
# Feed the data through two tower RNN, simple RNNs to
# start. No LSTM or GRUs with a margin and triplet loss.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Assuming D is the dimensionality of your embeddings
# input_size = EMBEDDING_SIZE  # Dimensionality of the input
hidden_size = 256  # Size of the RNN's hidden layer
output_size = 64  # Size of the final outout
EMBEDDING_SIZE = 128
rnn_doc = nn.RNN(EMBEDDING_SIZE, hidden_size)
rnn_answers = nn.RNN(EMBEDDING_SIZE, hidden_size)

def triplet_loss(anchor, positive, negative, margin=1.0):
    print(anchor.shape, positive.shape, negative.shape)
    distance_positive = (anchor - positive).pow(2).sum(1)
    distance_negative = (anchor - negative).pow(2).sum(1)
    losses = torch.relu(distance_positive - distance_negative + margin)
    return losses.mean()

In [11]:
for epoch in range(1):
    for i, (
        _,
        query_embedding,
        relevant_docs_embeddings,
        irrelevant_docs_embeddings,
    ) in enumerate(training_triples):
        # optimizer.zero_grad()
        # print(query_embedding.shape)
        # print(len(relevant_docs_embeddings))
        # print(irrelevant_docs_embeddings[0].shape)
        # print('.')
        # RUn the relevant examples through the rnn_answer
        # Run the query through the rnn_doc
        # Calculate the loss
        # Backpropagate
        # Optimize
        # query_output, _ = rnn_doc(1, 1, query_embedding)
        losses = []
        # stack them as T*B*E, where T is sequence length, B = 1, E = embedding size
        relevant_docs_embeddings_tensor = (
            torch.tensor(relevant_docs_embeddings)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(len(relevant_docs_embeddings), 1, -1)
        )
        # print(rd_view.shape)
        irrelevant_docs_embeddings_tensor = (
            torch.tensor(irrelevant_docs_embeddings)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(len(irrelevant_docs_embeddings), 1, -1)
        )
        # print(ird_view.shape)
        query_embedding_tensor = (
            torch.tensor(query_embedding)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(1, 1, -1)
        )
        print(relevant_docs_embeddings_tensor.shape, irrelevant_docs_embeddings_tensor.shape, query_embedding_tensor.shape)
        # query_output, _ = rnn_doc(query_embedding_tensor)
        # pos_doc_output, _ = rnn_answers(relevant_docs_embeddings_tensor)
        # neg_doc_output, _ = rnn_answers(irrelevant_docs_embeddings_tensor)
        # print(query_output.shape, pos_doc_output.shape, neg_doc_output.shape)
        # 1. I want to horizontally stack the quer_outoput pos_doc_output and neg_doc_output doc
        # 2. pass it through a linear layer
        
        # loss = triplet_loss(query_output, pos_doc_output, neg_doc_output)
        # losses.append(loss.item())
        # loss.backward()
        # optimizer.step()

        # for pos_doc_emb, neg_doc_emb in zip(relevant_docs_embeddings, irrelevant_docs_embeddings):
        #     pos_doc_output = rnn_answers(pos_doc_emb.unsqueeze(0).unsqueeze(0))  # Adjust dimensions if necessary
        #     neg_doc_output = doc_rnn(neg_doc_emb.unsqueeze(0))  # Adjust dimensions if necessary
        # loss = triplet_loss(query_output, pos_doc_output, neg_doc_output)
        # losses.append(loss)

        if i > 1:
            break
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {np.mean(losses)}")

torch.Size([7, 1, 128]) torch.Size([7, 1, 128]) torch.Size([1, 1, 128])
torch.Size([6, 1, 128]) torch.Size([6, 1, 128]) torch.Size([1, 1, 128])
torch.Size([6, 1, 128]) torch.Size([6, 1, 128]) torch.Size([1, 1, 128])
Epoch 0, Loss: nan


/tmp/ipykernel_93/147821194.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.tensor(relevant_docs_embeddings)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [17]:

relevant_docs_embeddings

[array([ 0.5593741 , -0.33353296,  0.26252204,  0.1313331 ,  0.07471806,
        -0.45851064, -0.45176926,  0.0285639 ,  0.7460716 , -0.3374388 ,
        -0.2323455 , -0.48056147, -0.48848403,  0.88539785, -0.03394746,
        -0.09290174, -0.1013684 ,  0.19646806,  0.02199814, -0.36809427,
         0.7379239 , -0.16938373, -0.3224192 , -0.17044698, -0.6510457 ,
        -0.27536213,  1.0406569 , -0.07615824,  0.29862505,  0.32756653,
         0.16935751, -0.15196942, -0.4551606 , -0.69615746, -0.5307793 ,
        -0.63438654, -0.48400837, -0.04019638, -0.2707593 , -0.7220823 ,
         0.32262215, -0.52835286, -0.11924949,  0.57100624,  0.13286802,
        -0.6309214 , -0.7163119 , -0.09836612,  0.2774239 , -0.3458099 ,
        -0.7920212 ,  1.2179525 ,  0.5729041 ,  0.03154727, -0.8517643 ,
        -0.30514222,  0.45712203, -0.78061384,  0.2688967 ,  0.19636531,
        -0.23007303, -0.04287523, -0.41295007, -0.32719812, -0.00900623,
        -0.61245453, -0.7482068 , -0.01362383,  0.0